# Determining popular posts on Hacker News

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.
[Data](https://www.kaggle.com/hacker-news/hacker-news-posts) has been picked up from [Kaggle](https://www.kaggle.com/) ,however it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.Documentation for the dataset can be found [here](https://www.kaggle.com/hacker-news/hacker-news-posts).

We are specifically interested in posts whose titles begin with either Ask HN or Show HN. We'll compare these two types of posts to determine :
+ Do Ask HN or Show HN receive more comments on average.
+ Do posts created at a certain time receive more comments on average.

### Opening and exploring the data

In [1]:
from csv import reader
opened_file=open('hacker_news.csv')
read_file=reader(opened_file)
hn=list(read_file)

def explore_data(dataset,start,end,rows_and_columns=False):
    for row in dataset[start:end]:
        print(row)
    if rows_and_columns == True:
        print('Number of columns=',len(dataset[0]))
        print('Number of rows=',len(dataset))
        
explore_data(hn,0,5,True)
    

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']
Number of columns= 7
Number of rows= 20101


We'll now store the header of the file in a separate variable and remove it from the dataset for the ease of working with the dataset.

In [2]:
header=hn[0]
hn=hn[1:]
print(header)
explore_data(hn,0,5,True)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']
['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']
Number of columns= 7
N

# Extracting Required Data

In [3]:
ask_posts=[]
show_posts=[]
other_posts=[]

for row in hn:
    title=row[1]
    title=title.lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print('Number of Ask HN posts',len(ask_posts))
print('Number of Show HN posts',len(show_posts))
print('Number of Other HN posts',len(other_posts))

Number of Ask HN posts 1744
Number of Show HN posts 1162
Number of Other HN posts 17194


## Calculating the average number of comments per post for Ask HN and Show HN posts

We'll calculate the average nuber of comments per post by iterating over the ask_posts data set to find the total number of comments for those posts and then dividing it by the total number of posts.

In [4]:
def column_avg(dataset,index):
    total=0
    for row in dataset:
        num=int(row[index])
        total += num
        
    return total/len(dataset)

avg_ask_comments=column_avg(ask_posts,4)
print(avg_ask_comments)

avg_show_comments=column_avg(show_posts,4)
print(avg_show_comments)

14.038417431192661
10.31669535283993


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

## Determining the time of the day for which the posts created recieve maximum comments

Next, we'll determine if we can maximize the amount of comments an ask post receives by creating it at a certain time. First, we'll find the amount of ask posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments ask posts created at each hour of the day receive.

In [5]:
import datetime as dt
for row in ask_posts:
    date=row[-1]
    date=dt.datetime.strptime(date,"%m/%d/%Y %H:%M")
    row[-1]=date
    
explore_data(ask_posts,0,5)

['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', datetime.datetime(2016, 8, 16, 9, 55)]
['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', datetime.datetime(2015, 11, 22, 13, 43)]
['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', datetime.datetime(2016, 5, 2, 10, 14)]
['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', datetime.datetime(2016, 8, 2, 14, 20)]
['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', datetime.datetime(2015, 10, 15, 16, 38)]


In [6]:
count_by_hour={}
comments_by_hour={}

for row in ask_posts:
    num_comments=int(row[4])
    date = row[-1]
    time=date.hour
    if time not in count_by_hour:
        count_by_hour[time]=1
    else :
        count_by_hour[time] += 1
        
        
    if time not in comments_by_hour:
        comments_by_hour[time]= num_comments
    else:
        comments_by_hour[time] += num_comments
        

        

We'll now calculate the average comments for each hour of the day by accessing the total number of comments in an hour and the total number of posts in that hour by using the dictionaries we created above.

In [18]:
avg_by_hour=[]
for hour in count_by_hour:
    avg_by_hour.append([hour,comments_by_hour[hour]/count_by_hour[hour]])
    
explore_data(avg_by_hour,0,5,True)

[9, 5.5777777777777775]
[13, 14.741176470588234]
[10, 13.440677966101696]
[14, 13.233644859813085]
[16, 16.796296296296298]
Number of columns= 2
Number of rows= 24


## Sorting and Printing Data 

We'll now sort the data by highest average number of comments.

In [17]:
sorted_by_hour=sorted(avg_by_hour,key=lambda x:x[1],reverse=True)
explore_data(sorted_by_hour,0,5)

[15, 38.5948275862069]
[2, 23.810344827586206]
[20, 21.525]
[16, 16.796296296296298]
[21, 16.009174311926607]


We'll now print the results in a readable form using string.format()

In [20]:
for row in sorted_by_hour:
    template="{hour}:00 : {avg:.2f} average comments per post"
    output=template.format(hour=row[0],avg=row[1])
    print(output)

15:00 : 38.59 average comments per post
2:00 : 23.81 average comments per post
20:00 : 21.52 average comments per post
16:00 : 16.80 average comments per post
21:00 : 16.01 average comments per post
13:00 : 14.74 average comments per post
10:00 : 13.44 average comments per post
14:00 : 13.23 average comments per post
18:00 : 13.20 average comments per post
17:00 : 11.46 average comments per post
1:00 : 11.38 average comments per post
11:00 : 11.05 average comments per post
19:00 : 10.80 average comments per post
8:00 : 10.25 average comments per post
5:00 : 10.09 average comments per post
12:00 : 9.41 average comments per post
6:00 : 9.02 average comments per post
0:00 : 8.13 average comments per post
23:00 : 7.99 average comments per post
7:00 : 7.85 average comments per post
3:00 : 7.80 average comments per post
4:00 : 7.17 average comments per post
22:00 : 6.75 average comments per post
9:00 : 5.58 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set documentation, the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

# Conclusion

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.